In [ ]:
import torch
print(torch.cuda.is_available() , torch.__version__)

In [1]:
from src2.api import Trainer  
app = Trainer.initialize(stage = 0 , resume = 0 , checkname= 1)

ModuleNotFoundError: No module named 'src2.model.util.basic'

In [ ]:
app.go()

NameError: name 'app' is not defined

In [ ]:
from src.api import HiddenExtractor
extractor = HiddenExtractor('resnet_gru_30m' , model_nums=[1] , model_types=['best'])
extractor.extract_hidden('update' , deploy = True)

In [ ]:
from src.api import DataAPI , Trainer
DataAPI.reconstruct_train_data()
Trainer.update_models()

In [ ]:
from src.api import Trainer
Trainer.update_models()